<a href="https://colab.research.google.com/github/ArshT/Pytorch_Practice/blob/master/Pytorch_FasionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as  nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5),(0.5))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=32,shuffle = True,num_workers = 2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset,batch_size=32,shuffle = False,num_workers = 2)

In [ ]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')

device = torch.device("cuda:0" if USE_GPU else "cpu")

using device: cuda


In [ ]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(1,32,3,(1,1),padding=(1,1))
    self.conv2 = nn.Conv2d(32,32,3,(1,1),padding=(1,1))
    self.batch1 = nn.BatchNorm2d(32)

    self.conv3 = nn.Conv2d(32,64,3,(1,1),padding=(1,1))
    self.conv4 = nn.Conv2d(64,64,3,(1,1),padding=(1,1))
    self.batch2 = nn.BatchNorm2d(64)

    self.pool = nn.MaxPool2d(2,2)
    self.drop = nn.Dropout(0.2)

    self.fc1 = nn.Linear(64*7*7,512)
    self.fc2 = nn.Linear(512,256)
    self.fc3 = nn.Linear(256,128)
    self.fc4 = nn.Linear(128,10)

  def forward(self,x):

    x = F.relu(self.batch1(self.conv1(x)))
    x = F.relu(self.batch1(self.conv2(x)))
    x = self.pool(x)

    x = F.relu(self.batch2(self.conv3(x)))
    x = F.relu(self.batch2(self.conv4(x)))
    x = self.pool(x)

    x = x.view(x.shape[0],64*7*7)

    x = F.relu(self.fc1(x))
    x = self.drop(x)
    x = F.relu(self.fc2(x))
    x = self.drop(x)
    x = F.relu(self.fc3(x))
    x = self.drop(x)
    x = self.fc4(x)

    return x

net = Net()
net = net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.003)

In [ ]:
def model_eval(dataloader):
    total = 0
    correct = 0
    for data in dataloader:
      images, l = data
      images = images.to(device)
      l = l.to(device)

      out = net(images)
      max_val, preds = torch.max(out,dim=1)
      
      total += l.shape[0]                   
      correct += (preds == l).sum().item()  
      accuracy = (100 * correct)/total
    
    
    return accuracy 

In [ ]:
for epoch in range(30):

  running_loss = 0.0
  for i,data in enumerate(trainloader,0):

    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if i % 300 == 299:    
      print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 300))
      running_loss = 0.0
  
  train_acc = model_eval(trainloader)
  test_acc = model_eval(testloader)

  print("Training Accuracy:",train_acc,"Testing Accuracy",test_acc)
  print("")

[1,   300] loss: 0.830
[1,   600] loss: 0.563
[1,   900] loss: 0.474
[1,  1200] loss: 0.407
[1,  1500] loss: 0.395
[1,  1800] loss: 0.375
Training Accuracy: 88.49333333333334 Testing Accuracy 87.43

[2,   300] loss: 0.318
[2,   600] loss: 0.326
[2,   900] loss: 0.319
[2,  1200] loss: 0.336
[2,  1500] loss: 0.304
[2,  1800] loss: 0.298
Training Accuracy: 90.51833333333333 Testing Accuracy 89.19

[3,   300] loss: 0.285
[3,   600] loss: 0.266
[3,   900] loss: 0.275
[3,  1200] loss: 0.267
[3,  1500] loss: 0.266
[3,  1800] loss: 0.283
Training Accuracy: 92.175 Testing Accuracy 90.86

[4,   300] loss: 0.234
[4,   600] loss: 0.234
[4,   900] loss: 0.237
[4,  1200] loss: 0.270
[4,  1500] loss: 0.256
[4,  1800] loss: 0.262
Training Accuracy: 92.45833333333333 Testing Accuracy 90.73

[5,   300] loss: 0.227
[5,   600] loss: 0.224
[5,   900] loss: 0.229
[5,  1200] loss: 0.240
[5,  1500] loss: 0.245
[5,  1800] loss: 0.229
Training Accuracy: 92.9 Testing Accuracy 90.23

[6,   300] loss: 0.199
[6,   

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
PATH = '/content/gdrive/My Drive/Pytorch_Practice/FashionMNIST_Model-1.pt'
torch.save(net,PATH)